In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

%matplotlib inline
plt.style.use('ggplot')



In [2]:
# notes_pd = pd.read_csv("/scratch/gobi1/johnchen/new_git_stuff/multimodal_fairness/data/physionet.org/files/mimiciii/1.4/NOTEEVENTS.csv")

In [3]:
# notes_pd[notes_pd["HADM_ID"] == 111]

In [4]:
# del notes_pd

In [5]:
import psycopg2
user = 'johnchen'
password = 'password'
dbname = 'mimic'
schema = 'mimiciii'
host = 'mimic'
# writing review papers such as: https://arxiv.org/pdf/1911.03977.pdf
query_schema = 'SET search_path to ' + schema + ';'

con = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

# Let's see what we're dealing with 
For now, do this in PSQL prompt. 


In [7]:
query = query_schema + """
SELECT count(icustay_id) as icu_counts
FROM icustays
group by hadm_id
ORDER BY icu_counts DESC
"""
df = pd.read_sql_query(query, con)
df.head()


,icu_counts
0,7
1,7
2,6
3,6
4,6


In [8]:
query = query_schema + """
SELECT count(hadm_id) as hadm_counts
FROM icustays
group by icustay_id
ORDER BY hadm_counts DESC
"""
df = pd.read_sql_query(query, con)
df.head()


,hadm_counts
0,1
1,1
2,1
3,1
4,1


In [ ]:
query = query_schema + """
SELECT *
FROM noteevents
"""
df = pd.read_sql_query(query, con)
print(len(df))

In [15]:
query = query_schema + """
SELECT *
FROM noteevents
where HADM_ID is NULL
"""
df = pd.read_sql_query(query, con)
df.head()

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,59660,82208,None,2200-01-02,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: Congest...
1,59669,15472,None,2176-05-28,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: Left ve...
2,59770,96774,None,2174-03-31,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: Dyspnea...
3,59805,1136,None,2193-09-19,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: Pericar...
4,59807,98046,None,2198-09-13,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: s/p AVR...


In [16]:
df.count()

row_id         231836
subject_id     231836
hadm_id             0
chartdate      231836
charttime      147149
storetime        5859
category       231836
description    231836
cgid             5859
iserror             7
text           231836
dtype: int64

# Now, we will try and see if we can join the notes back

In [9]:
query = query_schema + """
SELECT *
FROM noteevents
where SUBJECT_ID=36
"""
df = pd.read_sql_query(query, con)
df.head()


,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,7412,36,182104.0,2131-05-08,NaT,NaT,Discharge summary,Report,NaN,None,Admission Date: [**2131-4-30**] ...
1,7413,36,122659.0,2131-05-25,NaT,NaT,Discharge summary,Report,NaN,None,Admission Date: [**2131-5-12**] ...
2,7414,36,165660.0,2134-05-20,NaT,NaT,Discharge summary,Report,NaN,None,Admission Date: [**2134-5-10**] ...
3,67800,36,165660.0,2134-05-12,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: Coronar...
4,67801,36,122659.0,2131-05-19,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: history...


In [12]:
mapping_dict = {182104: 1, 122659:2}

In [13]:
df["new_col"] = df["hadm_id"].map(mapping_dict)

In [14]:
df

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text,new_col
0,7412,36,182104.0,2131-05-08,NaT,NaT,Discharge summary,Report,NaN,None,Admission Date: [**2131-4-30**] ...,1.0
1,7413,36,122659.0,2131-05-25,NaT,NaT,Discharge summary,Report,NaN,None,Admission Date: [**2131-5-12**] ...,2.0
2,7414,36,165660.0,2134-05-20,NaT,NaT,Discharge summary,Report,NaN,None,Admission Date: [**2134-5-10**] ...,NaN
3,67800,36,165660.0,2134-05-12,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: Coronar...,NaN
4,67801,36,122659.0,2131-05-19,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: history...,2.0
5,67802,36,122659.0,2131-05-18,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: Rising ...,2.0
6,67803,36,122659.0,2131-05-17,NaT,NaT,Echo,Report,NaN,None,"PATIENT/TEST INFORMATION:\nIndication: shock, ...",2.0
7,67804,36,122659.0,2131-05-17,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: H/O car...,2.0
8,67805,36,182104.0,2131-05-04,NaT,NaT,Echo,Report,NaN,None,PATIENT/TEST INFORMATION:\nIndication: Chest p...,1.0
9,141175,36,165660.0,2134-05-20,NaT,NaT,ECG,Report,NaN,None,Sinus rhythm. Normal tracing. Since the previo...,NaN


In [11]:
df.iloc[0]["text"]

'Admission Date:  [**2131-4-30**]              Discharge Date:   [**2131-5-8**]\n\nDate of Birth:  [**2061-8-17**]             Sex:   M\n\nService: CARDIOTHORACIC\n\nAllergies:\nPatient recorded as having No Known Allergies to Drugs\n\nAttending:[**First Name3 (LF) 922**]\nChief Complaint:\npositive stress test ; chest pain during hypertension to 220/110\n\n\nMajor Surgical or Invasive Procedure:\nCardiac Catheterization [**2131-4-30**]\nCABGx5 (LIMA-LAD,SVG-Diag,SVG-OM1,SVG-OM2,SVG-PDA)[**5-4**]\n\n\nHistory of Present Illness:\nMr. [**Known lastname 27218**] is a 69 yo M w/ h/o HTN and significant tobacco\nuse, who had a severe headache on [**2131-4-19**] without associated\nsymptoms (including weakness, visual changes), and was found to\nbe hypertensive to 220/110.  He was transferred via ambulence to\n[**Hospital 1474**] Hospital and in route complained of right sided chest\ndiscomfort, associated with dyspnea and diaphoreiss. By report\nhe was administered SL NTG and nitro paste w

In [10]:
len(df)

89

In [60]:
query = query_schema + """
SELECT *
FROM noteevents
where SUBJECT_ID=28090 and HADM_ID=121590 and category='Discharge summary'
"""
df = pd.read_sql_query(query, con)
display(df)

'''
Justify decisions that we made regarding preprocessing, and methodology
Fairness summary
Non-causal leakage for Decompensation
Timestamp needed for decomp prediction causal leakage
Actual modelling approaches:
Truncated back prop
Windows approach 

0. BOW
1. "Reg" NN (CNN from clinical/ rnn with truncated backprop); 
2. BERT
'''

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,33813,28090,121590,2187-05-18,None,None,Discharge summary,Report,None,None,Admission Date: [**2187-5-9**] D...


'\nJustify decisions that we made regarding preprocessing, and methodology\n'

# Try using the preprocessing used by Aggar..

In [61]:
import re
import json

from nltk import sent_tokenize, word_tokenize


In [62]:
SECTION_TITLES = re.compile(
    r'('
    r'ABDOMEN AND PELVIS|CLINICAL HISTORY|CLINICAL INDICATION|COMPARISON|COMPARISON STUDY DATE'
    r'|EXAM|EXAMINATION|FINDINGS|HISTORY|IMPRESSION|INDICATION'
    r'|MEDICAL CONDITION|PROCEDURE|REASON FOR EXAM|REASON FOR STUDY|REASON FOR THIS EXAMINATION'
    r'|TECHNIQUE'
    r'):|FINAL REPORT',
    re.I | re.M)


def pattern_repl(matchobj):
    """
    Return a replacement string to be used for match object
    """
    return ' '.rjust(len(matchobj.group(0)))


def find_end(text):
    """Find the end of the report."""
    ends = [len(text)]
    patterns = [
        re.compile(r'BY ELECTRONICALLY SIGNING THIS REPORT', re.I),
        re.compile(r'\n {3,}DR.', re.I),
        re.compile(r'[ ]{1,}RADLINE ', re.I),
        re.compile(r'.*electronically signed on', re.I),
        re.compile(r'M\[0KM\[0KM')
    ]
    for pattern in patterns:
        matchobj = pattern.search(text)
        if matchobj:
            ends.append(matchobj.start())
    return min(ends)


def split_heading(text):
    """Split the report into sections"""
    start = 0
    for matcher in SECTION_TITLES.finditer(text):
        # add last
        end = matcher.start()
        if end != start:
            section = text[start:end].strip()
            if section:
                yield section

        # add title
        start = end
        end = matcher.end()
        if end != start:
            section = text[start:end].strip()
            if section:
                yield section

        start = end

    # add last piece
    end = len(text)
    if start < end:
        section = text[start:end].strip()
        if section:
            yield section


def clean_text(text):
    """
    Clean text
    """

    # Replace [**Patterns**] with spaces.
    text = re.sub(r'\[\*\*.*?\*\*\]', pattern_repl, text)
    # Replace `_` with spaces.
    text = re.sub(r'_', ' ', text)

    start = 0
    end = find_end(text)
    new_text = ''
    if start > 0:
        new_text += ' ' * start
    new_text = text[start:end]

    # make sure the new text has the same length of old text.
    if len(text) - end > 0:
        new_text += ' ' * (len(text) - end)
    return new_text


def preprocess_mimic(text):
    """
    Preprocess reports in MIMIC-III.
    1. remove [**Patterns**] and signature
    2. split the report into sections
    3. tokenize sentences and words
    4. lowercase
    """
    for sec in split_heading(clean_text(text)):
        for sent in sent_tokenize(sec):
            text = ' '.join(word_tokenize(sent))
            yield text.lower()

In [63]:
# there are 27 possible matches!

In [64]:
df["row_id"].shape

(1,)

In [65]:
for index, row in df.iterrows():
    gen_ =  preprocess_mimic(row["text"])
    for out in gen_:
        print(out)

admission date : discharge date : date of birth : sex : m service : cardiothoracic allergies : aspirin / ibuprofen / motrin attending : chief complaint : tracheobronchomalacia .
major surgical or invasive
procedure :
flexible bronchoscopy with bronchoalveolar lavage of the right lower lobe , right thoracotomy , right upper lobe wedge resection , thoracic tracheoplasty with mesh ; right mainstem bronchoplasty and bronchus intermedius bronchoplasty with mesh ; left mainstem bronchoplasty with mesh .
history of present illness : mr. is a 67-year-old gentleman who has had cough , dyspnea and recurrent pulmonary infections .
he was found to have severe tracheobronchomalacia on bronchoscopy .
he had a stent trial and responded favorably in terms of his cough and dyspnea .
past medical
history :
recurrent pneumonias , asthma , bph , sinusitis , status post left meniscectomy of the left knee , status post turp , status post three sinus surgeries , status post tonsillectomy , status post ankle 

In [30]:
query = query_schema + """
SELECT *
FROM icustays
"""
df = pd.read_sql_query(query, con)
df.head()


,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [3]:
# check_all_tables_query = "SELECT * FROM tables;"
# all_tables = pd.read_sql_query(check_all_tables_query,con)

In [4]:
query = query_schema + """
SELECT *
FROM icustays
LIMIT 10
"""
df = pd.read_sql_query(query, con)
df.head()


,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [5]:
query = query_schema + """
SELECT *
FROM icustays
where subject_id=42210 
"""
df = pd.read_sql_query(query, con)
df.head()


,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
0,42074,42210,117994,281393,metavision,SICU,SICU,33,33,2193-02-11 21:32:36,2193-02-19 17:36:09,7.8358


In [6]:
query = query_schema + """
SELECT *
FROM admissions
where subject_id=42210 
"""
df = pd.read_sql_query(query, con)
df.head()


,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,40490,42210,117994,2193-02-11 21:31:00,2193-02-25 17:50:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,SPAN,CATHOLIC,MARRIED,HISPANIC/LATINO - DOMINICAN,2193-02-11 18:39:00,2193-02-11 23:05:00,SUBARACHNIOD HEMORRHAGE,0,1
1,40491,42210,192837,2193-04-10 07:15:00,2193-04-10 22:14:00,None,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicaid,SPAN,CATHOLIC,MARRIED,HISPANIC/LATINO - DOMINICAN,NaT,NaT,CEREBRAL ANEURYSM/SDA,0,0


In [7]:
query = query_schema + """
SELECT *
FROM noteevents
where subject_id=42210 
"""
df = pd.read_sql_query(query, con)
df.head()


,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,39025,42210,117994.0,2193-02-25,NaT,NaT,Discharge summary,Report,NaN,None,Admission Date: [**2193-2-11**] ...
1,723774,42210,117994.0,2193-02-18,2193-02-18 05:31:00,2193-02-18 05:31:39,Nursing,Nursing Progress Note,19169.0,None,Subarachnoid hemorrhage (SAH)\n Assessment:\...
2,723996,42210,117994.0,2193-02-19,2193-02-19 06:01:00,2193-02-19 09:32:14,Physician,Intensivist Note,16145.0,None,"TITLE:\n SICU\n HPI:\n 45F w severe H/A,..."
3,723943,42210,117994.0,2193-02-19,2193-02-19 02:57:00,2193-02-19 02:57:23,Nursing,Nursing Progress Note,20581.0,None,Subarachnoid hemorrhage (SAH)\n Assessment:\...
4,723817,42210,117994.0,2193-02-18,2193-02-18 06:03:00,2193-02-18 08:35:13,Physician,Intensivist Note,16545.0,None,"SICU\n HPI:\n 45F w severe H/A, vomitting;..."


In [8]:
df.iloc[0]["text"]

"Admission Date:  [**2193-2-11**]              Discharge Date:   [**2193-2-25**]\n\nDate of Birth:  [**2147-6-3**]             Sex:   F\n\nService: NEUROSURGERY\n\nAllergies:\nPatient recorded as having No Known Allergies to Drugs\n\nAttending:[**First Name3 (LF) 78**]\nChief Complaint:\nheadache\n\nMajor Surgical or Invasive Procedure:\n[**2193-2-11**]: EVD placement\n\n\nHistory of Present Illness:\n45F known only for HTN (treated) found to have very severe h/a\nand vomitting on [**2-11**]. CT head at OSH shows SAH and patient is\ntransferred to [**Hospital1 18**] for further eval.\n\n\nPast Medical History:\nHTN\n\nSocial History:\nnon-contributory\n\nFamily History:\nnon-contributory\n\nPhysical Exam:\nOn Admission:\n127/77  55  15  100%\nSleepy but arousable; spanish speaking only;\nFollows simple commands w all 4 ext.\nTalks very minimally; able to tell her name but not location or\ntime.\nPERLA, EOMI;\nFace symetric;\nTongue midline;\nMoves all 4 ext symetrically and with full s

In [9]:
df.columns

Index(['row_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text'],
      dtype='object')

In [10]:
query = query_schema + """
SELECT *
FROM icustay_detail
LIMIT 10
"""
df = pd.read_sql_query(query, con)
df.head()


,subject_id,hadm_id,icustay_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity,ethnicity_grouped,admission_type,hospital_expire_flag,hospstay_seq,first_hosp_stay,intime,outtime,los_icu,icustay_seq,first_icu_stay
0,2,163353,243653,M,NaT,2138-07-17 19:04:00,2138-07-21 15:48:00,3.8639,0.0022,ASIAN,asian,NEWBORN,0,1,True,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0918,1,True
1,3,145834,211552,M,2102-06-14,2101-10-20 19:08:00,2101-10-31 13:58:00,10.7847,76.5268,WHITE,white,EMERGENCY,0,1,True,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,1,True
2,4,185777,294638,F,NaT,2191-03-16 00:28:00,2191-03-23 18:41:00,7.7590,47.8450,WHITE,white,EMERGENCY,0,1,True,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,1,True
3,5,178980,214757,M,NaT,2103-02-02 04:31:00,2103-02-04 12:15:00,2.3222,0.0005,ASIAN,asian,NEWBORN,0,1,True,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0844,1,True
4,6,107064,228232,F,NaT,2175-05-30 07:15:00,2175-06-15 16:00:00,16.3646,65.9407,WHITE,white,ELECTIVE,0,1,True,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,1,True


In [11]:
df.columns

# we will definitely need to join it with other variables
# for instance, we need glucose_mean and other variables!

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'dod', 'admittime',
       'dischtime', 'los_hospital', 'age', 'ethnicity', 'ethnicity_grouped',
       'admission_type', 'hospital_expire_flag', 'hospstay_seq',
       'first_hosp_stay', 'intime', 'outtime', 'los_icu', 'icustay_seq',
       'first_icu_stay'],
      dtype='object')

In [12]:
adult_icu_query = query_schema + """
SELECT * from icustays
"""

In [13]:
adult_icu = pd.read_sql_query(query, con)

In [14]:
adult_icu.head()

,subject_id,hadm_id,icustay_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity,ethnicity_grouped,admission_type,hospital_expire_flag,hospstay_seq,first_hosp_stay,intime,outtime,los_icu,icustay_seq,first_icu_stay
0,2,163353,243653,M,NaT,2138-07-17 19:04:00,2138-07-21 15:48:00,3.8639,0.0022,ASIAN,asian,NEWBORN,0,1,True,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0918,1,True
1,3,145834,211552,M,2102-06-14,2101-10-20 19:08:00,2101-10-31 13:58:00,10.7847,76.5268,WHITE,white,EMERGENCY,0,1,True,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,1,True
2,4,185777,294638,F,NaT,2191-03-16 00:28:00,2191-03-23 18:41:00,7.7590,47.8450,WHITE,white,EMERGENCY,0,1,True,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,1,True
3,5,178980,214757,M,NaT,2103-02-02 04:31:00,2103-02-04 12:15:00,2.3222,0.0005,ASIAN,asian,NEWBORN,0,1,True,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0844,1,True
4,6,107064,228232,F,NaT,2175-05-30 07:15:00,2175-06-15 16:00:00,16.3646,65.9407,WHITE,white,ELECTIVE,0,1,True,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,1,True


In [15]:
# join on hadam (the hospital admissions)
# check: joining in postgresql faster than joining in pandas

hadm_query = query_schema + \\

'''
SELECT * 
from ADMISSIONS
'''


In [16]:
hadm_data = pd.read_sql_query(hadm_query,con)

In [17]:
# we could try and get patient embeddings for the mimic patient as well! Since it seems like we will have to at least get 
# lookups for dealing with the categorical embeddings: religion embedding! 
# actually: in general, we can always get categorical embeddings! (automatically)
# but: categorical embedding for decision trees etc. (do some clustering by country etc.)

# cogsci: religion embedding! 

# even just a cleaning on the diagnosis column would be super useful! 


In [18]:
hadm_data

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,None,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,NaT,NaT,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,NaT,NaT,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,None,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,None,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaT,NaT,V-TACH,0,1
6,27,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,NaT,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,None,CATHOLIC,None,WHITE,NaT,NaT,NEWBORN,0,1
7,28,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,NaT,NaT,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,NaT,NaT,UNSTABLE ANGINA\CATH,0,1
9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,None,CATHOLIC,MARRIED,WHITE,NaT,NaT,STATUS EPILEPTICUS,1,1


In [19]:
lab_events_query = query_schema + \\

'''
SELECT * 
FROM LABEVENTS
LIMIT 100
'''

In [20]:
lab_events = pd.read_sql_query(lab_events_query,con) #this query will take a while to run! (probably due to memory size, more than anything)

In [21]:
lab_events.head() # we will need something that maps it back to itemid

,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag
0,281,3,None,50820,2101-10-12 16:07:00,7.39,7.39,units,None
1,282,3,None,50800,2101-10-12 18:17:00,ART,NaN,None,None
2,283,3,None,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,None
3,284,3,None,50804,2101-10-12 18:17:00,22,22.00,mEq/L,None
4,285,3,None,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve



ImportError: cannot import name 'plot_roc_curve'

In [ ]:
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer